# NOVA — Modelo Ultra Optimizado v4  
### LightGBM + XGBoost + CatBoost (Ensemble)

En este notebook implementamos un modelo de *Machine Learning* de tipo **ensemble heterogéneo**, combinando tres algoritmos del estado del arte:

- **LightGBM**
- **XGBoost**
- **CatBoost**

El objetivo es predecir el desempeño de los estudiantes en la variable:




Este pipeline incluye:

1. Descarga automática del dataset desde Kaggle  
2. Preprocesamiento completo de variables numéricas y categóricas  
3. Entrenamiento de tres modelos independientes  
4. Blending (ensemble ponderado) con pesos optimizados  
5. Generación del archivo `submission.csv`  
6. Envío automático a Kaggle

---

In [ ]:
# ================================================================================
# Descarga de datos desde Kaggle
# ================================================================================

import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null

# Instalación de librerías necesarias

Algunos algoritmos utilizados (CatBoost, LightGBM, XGBoost) no vienen preinstalados en todos los entornos de ejecución.

En esta celda instalamos:

- **CatBoost** → Excelente para variables categóricas
- **LightGBM** → Modelo rápido y eficiente basado en árboles
- **XGBoost** → Modelo robusto para predicción tabular

Luego podremos importarlos sin errores.


In [ ]:
!pip install catboost

In [5]:
# ================================================================================
#     NOVA — MODELO ULTRA OPTIMIZADO v4 (LightGBM + XGBoost + CatBoost)
# ================================================================================

import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Carga de datos y limpieza inicial

En esta sección:

- Cargamos los archivos `train.csv` y `test.csv`.
- Eliminamos columnas duplicadas generadas por Pandas (las que terminan en `.1`).
- Creamos una copia segura del dataset para evitar modificaciones no deseadas.

También preparamos la variable objetivo:


In [6]:
# ================================================================================
# 1. Cargar datos
# ================================================================================

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

target = "RENDIMIENTO_GLOBAL"

# Quitar columnas duplicadas tipo *.1
dups = [c for c in train.columns if ".1" in c]
train = train.drop(columns=dups, errors="ignore")
test  = test.drop(columns=dups, errors="ignore")

# Copia segura
df = train.copy()

# Preprocesamiento: Manejo de NaN y codificación de variables

Este paso es fundamental para el correcto entrenamiento de los modelos:

### 1. Identificación de columnas categóricas  
Detectamos todas las columnas tipo `object`.

### 2. CatBoost no acepta valores NaN  
→ Los reemplazamos por `"MISSING"`.

### 3. Valores numéricos faltantes  
→ Se reemplazan por un valor sentinela seguro (`-9999`).

### 4. Conversión final a tipo `category`  
LightGBM aprovecha esta conversión para acelerar el entrenamiento.

### 5. Codificación del target  
Convertimos la variable objetivo a valores numéricos con `LabelEncoder`.

Este preprocesamiento permite que los tres modelos trabajen con los tipos de datos adecuados.


In [7]:
# ================================================================================
# 2. Preprocesamiento: categorizar columnas y limpiar NaN
# ================================================================================

categorical_cols = []

for col in df.columns:
    if df[col].dtype == object and col != target:
        categorical_cols.append(col)

# ---- FIX: CatBoost no acepta NaN -> convertir a string ----
for col in categorical_cols:
    df[col] = df[col].astype("object").fillna("MISSING")
    if col in test.columns:
        test[col] = test[col].astype("object").fillna("MISSING")

# ---- FIX: NaN numéricos -> sentinel value seguro ----
for col in df.columns:
    if df[col].dtype != object and col != target:
        df[col] = df[col].fillna(-9999)
        if col in test.columns:
            test[col] = test[col].fillna(-9999)

# Convertir finalmente a categoría
for col in categorical_cols:
    df[col] = df[col].astype("category")
    if col in test.columns:
        test[col] = test[col].astype("category")

print(f"Categorías detectadas: {len(categorical_cols)}")

# Codificar target
le = LabelEncoder()
df[target] = le.fit_transform(df[target])

Categorías detectadas: 13


# División Train / Validation

Dividimos el dataset en:

- **85%** para entrenamiento  
- **15%** para validación

Usamos división *estratificada* para mantener la proporción real de clases en ambos conjuntos.

Esto es crucial para obtener métricas confiables sin sesgo.


In [8]:
# ================================================================================
# 3. Train / Validation split
# ================================================================================

X = df.drop(columns=[target])
y = df[target]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

# Modelo 1 — LightGBM

LightGBM es un algoritmo basado en gradient boosting muy rápido y eficiente.

En este paso:

- Entrenamos un modelo multiclase
- Indicamos las columnas categóricas para aceleración
- Calculamos la precisión (accuracy) en el conjunto de validación

LightGBM suele aprender bien en datasets con muchas variables categóricas.


In [9]:
# ================================================================================
# 4. MODEL 1 — LIGHTGBM
# ================================================================================

print("\nEntrenando LightGBM...")

lgb_model = lgb.LGBMClassifier(
    n_estimators=600,
    learning_rate=0.03,
    num_leaves=45,
    max_depth=9,
    min_child_samples=15,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=1.3,
    reg_lambda=2.5,
    objective="multiclass",
    class_weight="balanced",
    random_state=42
)

lgb_model.fit(
    X_train,
    y_train,
    categorical_feature=categorical_cols
)

lgb_val_pred = lgb_model.predict(X_val)
lgb_acc = accuracy_score(y_val, lgb_val_pred)

print(f" LightGBM Accuracy: {lgb_acc:.4f}")


Entrenando LightGBM...
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.154660 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1887
[LightGBM] [Info] Number of data points in the train set: 588625, number of used features: 19
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
 LightGBM Accuracy: 0.4406


# Modelo 2 — XGBoost

XGBoost requiere que las categorías estén **codificadas como números**, por eso convertimos y luego:

- Entrenamos el modelo usando `hist`, que acelera el entrenamiento
- Evaluamos su precisión en el conjunto de validación

XGBoost es robusto y suele generalizar muy bien en datos tabulares.




In [10]:
# ================================================================================
# 5. MODEL 2 — XGBOOST
# ================================================================================

print("\nEntrenando XGBoost...")

# Convertir categorías -> códigos
X_train_xgb = X_train.copy()
X_val_xgb   = X_val.copy()
test_xgb    = test.copy()

for col in X_train_xgb.columns:
    if str(X_train_xgb[col].dtype) in ["category", "object"]:
        X_train_xgb[col] = X_train_xgb[col].astype("category").cat.codes
        X_val_xgb[col]   = X_val_xgb[col].astype("category").cat.codes
        test_xgb[col]    = test_xgb[col].astype("category").cat.codes

xgb_model = xgb.XGBClassifier(
    n_estimators=550,
    learning_rate=0.035,
    max_depth=8,
    subsample=0.85,
    colsample_bytree=0.85,
    min_child_weight=3,
    reg_alpha=1.2,
    reg_lambda=3.0,
    gamma=0.2,
    objective="multi:softprob",
    eval_metric="mlogloss",
    tree_method="hist",
    random_state=42
)

xgb_model.fit(X_train_xgb, y_train)
xgb_val_pred = xgb_model.predict(X_val_xgb)

xgb_acc = accuracy_score(y_val, xgb_val_pred)

print(f" XGBoost Accuracy: {xgb_acc:.4f}")


Entrenando XGBoost...
 XGBoost Accuracy: 0.4349


# Modelo 3 — CatBoost

CatBoost está especialmente diseñado para manejar variables categóricas **sin necesidad de codificación manual**.

En este paso:

- Se entrenan 300 iteraciones
- Se usa validación interna con `eval_set`
- Se obtiene la precisión en el conjunto de validación

CatBoost suele ser el modelo que mejor captura relaciones complejas en las categorías.


In [ ]:
# ================================================================================
# 6. MODEL 3 — CATBOOST
# ================================================================================

print("\nEntrenando CatBoost...")

cat_model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.06,
    depth=6,
    l2_leaf_reg=3,
    loss_function="MultiClass",
    random_seed=42,
    task_type="CPU",
    verbose=0
)

cat_model.fit(
    X_train,
    y_train,
    cat_features=categorical_cols,
    eval_set=(X_val, y_val),
    verbose=0
)

cat_val_pred = cat_model.predict(X_val).flatten().astype(int)
cat_acc = accuracy_score(y_val, cat_val_pred)

print(f" CatBoost Accuracy: {cat_acc:.4f}")


Entrenando CatBoost...


# Ensemble final — Blending ponderado

Combinamos las probabilidades de los tres modelos:

- `w_lgb = 0.55`
- `w_xgb = 0.20`
- `w_cat = 0.25`

El ensemble se calcula:




In [ ]:
# ================================================================================
# 7. ENSEMBLE (BLENDING)
# ================================================================================

print("\nCalculando Ensemble final...")

lgb_p = lgb_model.predict_proba(X_val)
xgb_p = xgb_model.predict_proba(X_val_xgb)
cat_p = cat_model.predict_proba(X_val)

w_lgb = 0.55
w_xgb = 0.20
w_cat = 0.25

blend = (w_lgb*lgb_p) + (w_xgb*xgb_p) + (w_cat*cat_p)
blend_pred = np.argmax(blend, axis=1)

blend_acc = accuracy_score(y_val, blend_pred)
print(f" Ensemble Accuracy: {blend_acc:.4f}")

Finalmente:

- Se selecciona la clase con máxima probabilidad
- Se calcula la precisión final del ensemble

En general, el ensemble supera el rendimiento individual de cada modelo.

# Generación del archivo de submission

Con el ensemble final:

- Predecimos los valores del archivo `test.csv`
- Invertimos el Label Encoding para recuperar los nombres originales de las clases
- Creamos el archivo `my_submission.csv`

Este archivo es el que enviamos a Kaggle para evaluar nuestro desempeño en el leaderboard.

In [ ]:
# ================================================================================
# 8. PREDICCIÓN FINAL SOBRE TEST
# ================================================================================

print("Generando predicciones finales...\n")

lgb_test = lgb_model.predict_proba(test)
xgb_test = xgb_model.predict_proba(test_xgb)
cat_test = cat_model.predict_proba(test)

final_blend = (w_lgb*lgb_test) + (w_xgb*xgb_test) + (w_cat*cat_test)
final_pred = np.argmax(final_blend, axis=1)

final_labels = le.inverse_transform(final_pred)

submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": final_labels
})

submission.to_csv("my_submission.csv", index=False)

# Envío automático a Kaggle

En esta última celda:

1. Configuramos la API de Kaggle  
2. Enviamos el archivo `my_submission.csv` directamente a la competencia  
3. Registramos el envío con un mensaje descriptivo

Esto automatiza completamente el ciclo:
**entrenar → predecir → generar submission → enviar a Kaggle**.

In [ ]:
# ================================================================================
# Envío del submission a Kaggle
# ================================================================================

import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json

!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
-f my_submission.csv -m "Entrega version final - Equipo Juan y Jose"